In [1]:
import json

# Open the JSON file and load the data
with open('allDocument.json', 'r') as file:
    data = json.load(file)

# Print the loaded data
# print(len(data[0]))
data[0]


{'question': "In the case of Nasr v NRMA Insurance [2006] NSWSC 1018, why was the plaintiff's appeal lodged out of time?",
 'text': "In Nasr v NRMA Insurance [2006] NSWSC 1018, the plaintiff's appeal was lodged out of time because the summons was filed on 8 June 2006, seven months after the decision of the Local Court was made on 4 October 2005. No explanation was provided for this delay.",
 'doc_id': '81ceb3c0'}

In [2]:
from collections import defaultdict

In [3]:
hashes = defaultdict(list)

for doc in data:
    docId = doc['doc_id']
    hashes[docId].append(doc)

In [4]:
hashes['81ceb3c0']

[{'question': "In the case of Nasr v NRMA Insurance [2006] NSWSC 1018, why was the plaintiff's appeal lodged out of time?",
  'text': "In Nasr v NRMA Insurance [2006] NSWSC 1018, the plaintiff's appeal was lodged out of time because the summons was filed on 8 June 2006, seven months after the decision of the Local Court was made on 4 October 2005. No explanation was provided for this delay.",
  'doc_id': '81ceb3c0'}]

In [5]:
import ollama

promptTemplate = """
    You emulate a user who will use the legal document assistant.
    Formulate 3 questions this user might ask based on a legal documents record. The record
    should contain the answer to the questions, and the questions should be complete and not too short.
    If possible, use as fewer words as possible from the record. 

    The record:
    question: {question}
    text: {text}

    Provide the output in array like this format below:

    ["question1", "question2", ..., "question5"]
    """.strip()

def generate_questions(doc):
    prompt = promptTemplate.format(**doc)

    llm = ollama.chat(
        model="llama2:7b",
        messages=[{"role": "user", "content":prompt}],
        # stream=True
    )

    
    return llm['message']['content']



In [6]:
from tqdm.auto import tqdm 

C:\Users\Lenovo\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
results = {}
for doc in tqdm(data): 
    doc_id = doc['doc_id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions
    # print(questions)

100%|██████████| 50/50 [2:33:20<00:00, 184.02s/it]  


In [8]:
print(results['81ceb3c0'])


Sure! Here are three questions that a user who has accessed the legal document assistant might ask based on the record provided:

["Why did the plaintiff wait seven months to file an appeal?", "Was there a specific reason for the delay in filing the appeal?", "What were the consequences of filing the appeal out of time?"]


In [9]:
results

{'81ceb3c0': '\nSure! Here are three questions that a user who has accessed the legal document assistant might ask based on the record provided:\n\n["Why did the plaintiff wait seven months to file an appeal?", "Was there a specific reason for the delay in filing the appeal?", "What were the consequences of filing the appeal out of time?"]',
 '87bce545': '\nSure! Here are 3 questions that the user might ask based on the legal document record:\n\n["What was the Appellant\'s relationship with Mr. Nguyen?", "What activities did the Appellant witness Mr. Nguyen engaging in at the flat?", "Why did the Appellant take no further action to prevent Mr. Nguyen from selling heroin despite her initial protests?"]',
 '07111b79': '\nSure! Here are three questions that the user might ask based on the legal document record:\n\n["What was the court\'s reasoning for dismissing the motion to restrain Scenic Tours Pty Ltd from acting?", "Did the plaintiff have a choice in selecting their solicitor?", "Was

In [13]:
all_results = results

In [14]:
import pickle

with open('results.bin', 'rb') as f_in:
    all_results = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: 'results.bin'

In [10]:
import pandas as pd

In [15]:
doc_index = {d['doc_id']: d for d in data}

In [31]:
import ast

final_results = []

for doc_id, questions in all_results.items():
        # Extract the list portion from the input string
        try:
            start_index = questions.find("[")
            end_index = questions.find("]") + 1
            list_string = questions[start_index:end_index]
            questions_list = ast.literal_eval(list_string)
            for q in questions_list:
                final_results.append((q,doc_id))
        except Exception as e:
            print(e)
            continue

EOL while scanning string literal (<unknown>, line 1)
EOL while scanning string literal (<unknown>, line 1)
EOL while scanning string literal (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
unexpected EOF while parsing (<unknown>, line 0)
invalid syntax (<unknown>, line 1)
EOL while scanning string literal (<unknown>, line 1)
EOL while scanning string literal (<unknown>, line 1)


In [32]:
df = pd.DataFrame(final_results, columns=['question', 'document'])
df.to_csv('ground-truth-data.csv', index=False)